
# N2 — Modelagem (Painel de Controle) — **Patch de caminhos aplicado**

- Corrigido: criação de `reports/` e `artifacts/` no diretório dos notebooks.  
- Agora o N2 **descobre a raiz do projeto** pelo `config/defaults.json` e usa essa raiz para todos os caminhos.  
- `data_processed_file` é uma **chave de config** (não é função). Se você definir o nome do arquivo lá, o N2 usa direto.


# 📘 Bootstrap do N2 — Raiz do Projeto, Configuração e Dataset

Nesta etapa inicial, o notebook realiza o **bootstrap** do ambiente de modelagem, preparando todo o contexto necessário para o treinamento de modelos supervisionados.

## Principais ações executadas

1. **Localização automática da raiz do projeto (`PROJECT_ROOT`)**
   O código sobe a árvore de diretórios até encontrar o arquivo `config/defaults.json`.
   Esse processo garante que o notebook funcione corretamente mesmo se for aberto a partir de subpastas (como `notebooks/`).

2. **Injeção do caminho da raiz e da pasta `utils/` no `sys.path`**
   Essa etapa permite que os módulos auxiliares do projeto (`utils/utils_data.py`) sejam importados normalmente, sem precisar ajustar manualmente os caminhos no ambiente.

3. **Carregamento da configuração global (`defaults.json`)**
   Todas as definições de comportamento — como colunas-alvo, escalonamento, proporção de teste, regras de outliers e encoding — são lidas diretamente desse arquivo centralizado, assegurando consistência entre N1, N2 e N3.

4. **Garantia dos diretórios padrão**
   As pastas `artifacts/`, `reports/` e `artifacts/models/` são criadas automaticamente, caso não existam, organizando as saídas do pipeline de modelagem.

5. **Descoberta do dataset processado**
   O notebook identifica e carrega o arquivo final preparado no N1, normalmente localizado em `data/processed/processed.parquet`.
   Caso não o encontre, uma mensagem orienta o usuário a revisar o `defaults.json` ou reexecutar o N1.

6. **Leitura do dataset e definição da variável alvo (`TARGET_COL`)**
   O dataset é carregado conforme o formato detectado (Parquet, CSV ou Excel).
   Em seguida, a variável alvo é localizada automaticamente com base na configuração, e as variáveis independentes (`X`) são separadas.

7. **Resumo rápido do ambiente e do dataset**
   Ao final, são exibidas informações gerais sobre o projeto, o arquivo carregado, dimensões, memória, contagem de colunas por tipo, nulos e distribuição da variável alvo — criando um panorama rápido e auditável do ponto de partida para a modelagem.

---

> **Em resumo:**  
> Esta célula garante que o N2 sempre inicie em um ambiente limpo, conectado à raiz do projeto, com acesso às utilidades centralizadas, configuração carregada e dataset processado prontos para o treinamento dos modelos.


In [1]:
# -*- coding: utf-8 -*-
from __future__ import annotations

import sys
import json
import random
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, Tuple

import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    RocCurveDisplay,
    ConfusionMatrixDisplay,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import ipywidgets as W
from IPython.display import display, clear_output

np.random.seed(42)
random.seed(42)

# ------------------------------
# Bootstrap de caminho: encontra raiz e injeta no sys.path
# ------------------------------
def _find_up(relative_path: str, start: Path | None = None) -> Path | None:
    start = start or Path.cwd()
    rel = Path(relative_path)
    for base in (start, *start.parents):
        cand = base / rel
        if cand.exists():
            return cand
    return None

_cfg = _find_up("config/defaults.json")
if _cfg is None:
    raise FileNotFoundError(
        "config/defaults.json não encontrado. Abra o notebook N2 dentro da estrutura do projeto."
    )

PROJECT_ROOT = _cfg.parent.parent  # .../config/defaults.json -> raiz
# injeta raiz e pasta utils no sys.path (para permitir `from utils...`)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
utils_dir = PROJECT_ROOT / "utils"
if utils_dir.exists() and str(utils_dir) not in sys.path:
    sys.path.insert(0, str(utils_dir))

print("[INFO] PROJECT_ROOT:", PROJECT_ROOT)
print("[INFO] sys.path inclui raiz?", str(PROJECT_ROOT) in sys.path)
print("[INFO] sys.path inclui utils?", str(utils_dir) in sys.path)

# >>>>> utilidades agora vêm do utils_data <<<<<
from utils.utils_data import (
    get_project_root, load_config, ensure_dirs, discover_processed_path,
    summarize_columns, compute_metrics, try_plot_roc, persist_artifacts
)

# ------------------------------
# Carrega config, garante dirs e resolve dataset processado
# ------------------------------
cfg = load_config()
artifacts_dir, reports_dir, models_dir = ensure_dirs(cfg)
processed_path = discover_processed_path(cfg)
print("[INFO] Active config:", json.dumps(cfg, indent=2, ensure_ascii=False))
print("[INFO] Processed file:", processed_path)

# Leitura do dataset (suporte a parquet/csv/xlsx)
if processed_path.suffix.lower() in [".parquet", ".pq"]:
    df = pd.read_parquet(processed_path)
elif processed_path.suffix.lower() == ".csv":
    df = pd.read_csv(processed_path)
elif processed_path.suffix.lower() in [".xlsx", ".xls"]:
    df = pd.read_excel(processed_path)
else:
    raise ValueError(f"Extensão não suportada: {processed_path.suffix}")

# ------------------------------
# Função de pré-processamento — **fica exposta no notebook**
# ------------------------------
def build_preprocess(numeric_cols, categorical_cols, scale_numeric=True):
    num_steps = [("imputer", SimpleImputer(strategy="mean"))]
    if scale_numeric:
        num_steps.append(("scaler", StandardScaler()))
    numeric_transformer = Pipeline(steps=num_steps)
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False)),
    ])
    return ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_cols),
            ("cat", categorical_transformer, categorical_cols),
        ],
        remainder="drop",
        verbose_feature_names_out=False,
    )


# Confirmar leitura e target
df.head(3)

TARGET_COL = df.columns[df.columns.str.lower() == str(cfg.get("target_column", "target")).lower()]
TARGET_COL = TARGET_COL[0] if len(TARGET_COL) else cfg.get("target_column", "target")
print("[INFO] TARGET_COL =", TARGET_COL)
assert TARGET_COL in df.columns, f"Target '{TARGET_COL}' não encontrada no dataset."

# Separar X/y
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]
print(X.shape, y.shape)

# === Painel rápido de status do N2 (resumo explicativo) ===
from pathlib import Path

def _fmt_mb(n_bytes: int) -> str:
    return f"{n_bytes / (1024**2):.2f} MB"

def _pct(n, d) -> str:
    return f"{(100*n/d):.2f}%" if d else "n/a"

# 1) Info do dataset
n_rows, n_cols = df.shape
mem_bytes = df.memory_usage(deep=True).sum()

# 2) Tipos de colunas
num_cols, cat_cols, other_cols = summarize_columns(df)

# 3) Nulos (visão geral)
null_total = int(df.isna().sum().sum())
null_any_cols = int((df.isna().sum() > 0).sum())

# 4) Target
target_name = TARGET_COL
n_unique_target = int(pd.Series(y).nunique())
target_counts = y.value_counts(dropna=False)
target_pct = (target_counts / len(y) * 100).round(2)

# 5) Split config (sem executar o split ainda; apenas ecoa a config)
test_size = cfg.get("test_size", 0.2)
random_state = cfg.get("random_state", 42)
scale_numeric = bool(cfg.get("scale_numeric", True))

print("\n" + "="*78)
print("N2 — Resumo do bootstrap e leitura do dataset")
print("="*78)
print(f"• Projeto (PROJECT_ROOT): {PROJECT_ROOT}")
print(f"• Arquivo lido         : {processed_path}")
print(f"• Formato              : {processed_path.suffix.lower()}")
print(f"• Dimensão             : {n_rows} linhas × {n_cols} colunas")
print(f"• Memória estimada     : {_fmt_mb(mem_bytes)}")
print(f"• Colunas por tipo     : num={len(num_cols)} | cat={len(cat_cols)} | other={len(other_cols)}")
print(f"• Nulos (total)        : {null_total} células nulas em {null_any_cols} colunas com nulos")
print(f"• Target               : '{target_name}' (valores únicos={n_unique_target})")

# Distribuição da target (top 5 caso multiclasses extensas)
print("• Distribuição da target:")
if len(target_counts) > 5:
    to_show = target_counts.head(5)
    to_show_pct = target_pct.head(5)
    others = len(target_counts) - 5
    print((pd.DataFrame({'count': to_show, 'pct': to_show_pct}).to_string()))
    print(f"  ... (+{others} classes)")
else:
    print((pd.DataFrame({'count': target_counts, 'pct': target_pct}).to_string()))

print("-"*78)
print("Parâmetros previstos (antes do split):")
print(f"• test_size   = {test_size}")
print(f"• random_state= {random_state}")
print(f"• scale_numeric (pré-processamento) = {scale_numeric}")

[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] sys.path inclui raiz? True
[INFO] sys.path inclui utils? True
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [ensure_dirs] artifacts=C:\Users\fabio\Projetos DEV\data projects\data-project-template\artifacts | reports=C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports | models=C:\Users\fabio\Projetos DEV\data projects\data-project-template\artifacts\models
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] Active config: {
  "infer_types": true,
  "cast_numeric_like": true,
  "strip_whitespace": true,
  "handle_missing": true,
  "missing_strategy": "simple",
  "detect_outliers": true,
  "outlier_method": "iqr",
  "outliers": {
    "cols": null,
    "exclude_cols": [
      "customerID"
    ],
    "exclude_bin

# 🔄 Carregamento da Config e Descoberta do Dataset Processado

Nesta etapa, o N2 sincroniza o ambiente de modelagem com as **definições globais** do projeto e obtém o **dataset final** preparado no N1.

## O que acontece aqui

1. **Leitura da configuração (`defaults.json`)**
   - Centraliza parâmetros reutilizáveis entre N1, N2 e N3 (ex.: `target_column`, `test_size`, `random_state`, `scale_numeric`, regras de *feature engineering*).
   - Garante consistência entre projetos e evita "configuração espalhada" dentro dos notebooks.

2. **Garantia de diretórios de saída**
   - Cria (se necessário) `artifacts/`, `reports/` e `artifacts/models/` na **raiz do projeto**.
   - Mantém modelos, métricas e manifestos organizados em uma estrutura previsível.

3. **Descoberta do dataset processado**
   - Resolve automaticamente o arquivo de `data/processed` (preferindo `data_processed_file`; caso ausente, tenta por extensão).
   - Erros informativos orientam a ajustar o `defaults.json` ou reexecutar o N1, caso o arquivo não exista.

4. **Leitura robusta do dataset**
   - Suporte a Parquet/CSV/Excel, com *fallback* de engine para Parquet quando necessário.
   - Erros de leitura são encapsulados com mensagens claras, úteis para depuração.

5. **Determinação da variável alvo (`TARGET_COL`)**
   - Identifica a coluna alvo via `target_column` (ou `target.name`) de forma *case-insensitive*.
   - Se a coluna não existir, interrompe com instrução objetiva de correção.

6. **Separação de variáveis (X/y) e diagnóstico da target**
   - Remove a coluna alvo de `X` e mantém `y` com a distribuição original.
   - Exibe **número de nulos**, **nº de classes** e **distribuição** (contagem e %).
   - Caso exista `class_map`, informa a **cobertura potencial** do mapeamento (sem alterar `y`).

## Por que esta etapa é importante?

- Garante que o N2 está alinhado com a **configuração oficial** do projeto.
- Confere se o dataset final do N1 está **presente e íntegro** antes de prosseguir.
- Fornece um **resumo auditável** (formas de `X/y`, distribuição da target), útil para apresentações e *debug*.

---

> **Dica:** Se quiser normalizar a classe alvo (por exemplo, trocando *Yes/No* por 1/0), faça uma cópia (`y_mapped`) usando `class_map` **apenas** depois de validar a distribuição original.


In [2]:
# === Carregamento da config e descoberta do dataset processado (robusto) ===

cfg = load_config()
artifacts_dir, reports_dir, models_dir = ensure_dirs(cfg)

processed_path = discover_processed_path(cfg)
print("[INFO] Active config:", json.dumps(cfg, indent=2, ensure_ascii=False))
print("[INFO] Processed file:", processed_path)

# Leitura do dataset (robusta e informativa)
suffix = processed_path.suffix.lower()
try:
    if suffix in (".parquet", ".pq"):
        try:
            df = pd.read_parquet(processed_path, engine="pyarrow")
        except Exception:
            # fallback útil quando pyarrow não está disponível
            df = pd.read_parquet(processed_path, engine="fastparquet")
    elif suffix == ".csv":
        # low_memory=False evita dtypes quebrados; encoding utf-8 por padrão
        df = pd.read_csv(processed_path, low_memory=False, encoding="utf-8")
    elif suffix in (".xlsx", ".xls"):
        df = pd.read_excel(processed_path)
    else:
        raise ValueError(f"Extensão não suportada: {suffix}")
except Exception as e:
    raise RuntimeError(f"Falha ao ler '{processed_path.name}': {type(e).__name__}: {e}")

# Determinação tolerante da coluna alvo
cfg_target = (
    (cfg.get("target_column"))
    or (cfg.get("target", {}) or {}).get("name")
    or "target"
)
cands = [c for c in df.columns if c.lower() == str(cfg_target).lower()]
if not cands:
    raise KeyError(
        f"Target '{cfg_target}' não encontrada no dataset. "
        f"Defina corretamente 'target_column' (ou target.name) em config/defaults.json."
    )
TARGET_COL = cands[0]
print(f"[INFO] Target: {TARGET_COL}")

# Separar X/y
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Diagnóstico rápido da target
n_null = int(pd.isna(y).sum())
n_classes = int(pd.Series(y).nunique(dropna=True))
print(f"[CHECK] Target nulos={n_null} | classes únicas={n_classes}")
print("[CHECK] Distribuição da target (top 5):")
print((y.value_counts(dropna=False).head(5).to_frame("count")
         .assign(pct=lambda d: (d["count"]/len(y)*100).round(2))))

# (Opcional) Pré-visualização do shape de X/y
print(f"[INFO] Shapes -> X: {X.shape} | y: {y.shape}")

# (Opcional) Classe-alvo mapeada (somente mostra, não sobrescreve)
class_map = cfg.get("class_map") or {}
if class_map:
    # mapeamento case-insensitive para exibir se seria possível mapear
    inv = {str(k).strip().lower(): v for k, v in class_map.items()}
    preview = (y.astype(str).str.strip().str.lower()).map(inv)
    ok_ratio = preview.notna().mean()
    print(f"[INFO] class_map detectado. Mapeamento possível em {ok_ratio:.1%} das linhas.")
    if ok_ratio < 1.0:
        print("[AVISO] Há valores em y que não casam com class_map; revise 'class_map' no defaults.json.")


[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] [ensure_dirs] artifacts=C:\Users\fabio\Projetos DEV\data projects\data-project-template\artifacts | reports=C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports | models=C:\Users\fabio\Projetos DEV\data projects\data-project-template\artifacts\models
[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] Active config: {
  "infer_types": true,
  "cast_numeric_like": true,
  "strip_whitespace": true,
  "handle_missing": true,
  "missing_strategy": "simple",
  "detect_outliers": true,
  "outlier_method": "iqr",
  "outliers": {
    "cols": null,
    "exclude_cols": [
      "customerID"
    ],
    "exclude_binaries": true,
    "iqr_factor": 1.5,
    "z_threshold": 3.0,
    "persist_summary": true,
    "persist_relpath": "outliers/summary.csv"
  },
  "deduplicat

# ✂️ Split Treino/Teste e Resumo de Colunas

Nesta etapa, preparamos os dados para a modelagem supervisionada, separando variáveis independentes (`X`) da variável alvo (`y`) e realizando o **split treino/teste** com parâmetros consistentes definidos no `defaults.json`.

## O que acontece aqui

1. **Separação de X e y**
   - Remove-se a coluna alvo (`TARGET_COL`) de `X` e mantém-se `y` intacta para preservar a distribuição original.

2. **Resumo de tipos de colunas**
   - Gera contagens de colunas **numéricas**, **categóricas** e **outras** (se houver).
   - Este diagnóstico orienta a construção do `ColumnTransformer` na próxima etapa.

3. **Split treino/teste com estratificação**
   - Usa `test_size` e `random_state` da configuração.
   - Ativa `stratify=y` sempre que houver mais de uma classe, mantendo a **mesma proporção** de classes em `train` e `test`.

4. **Diagnósticos úteis**
   - Compara a **distribuição da target** no conjunto geral vs. `train` vs. `test` (contagem e %).
   - Sinaliza **desbalanceamento** quando a classe majoritária ultrapassa um limite (ex.: 80%).
   - Detecta **categorias raras** (freq. < 5 no `train`) em variáveis categóricas — útil para evitar explosão do One-Hot e para tratar identificadores.

## Observações frequentes

- **IDs como `customerID`** aparecem como inúmeras categorias raras. Em geral, **não devem ser usadas como preditores**, pois são identificadores sem relação causal. Recomenda-se **removê-las** de `X` (ou excluí-las da lista de categóricas antes do One-Hot).
- Se houver **alto cardinalidade** e a variável for preditiva, considere técnicas específicas (target encoding, hashing, catboost encoder). Neste template, mantemos a abordagem pedagógica e transparente com One-Hot e remoção de IDs.

## Próximo passo

- Construir o **pré-processamento** com `ColumnTransformer` (imputação + One-Hot para categóricas + escala opcional para numéricas) e ajustar no conjunto de treino.


In [3]:
# === Split treino/teste e resumo de colunas (diagnóstico reforçado) ===
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Resumo de colunas do X
num_cols, cat_cols, other_cols = summarize_columns(X)
print(f"[INFO] Colunas numéricas: {len(num_cols)} | categóricas: {len(cat_cols)} | ignoradas: {len(other_cols)}")

# Parâmetros do split (eco explícito)
test_size = float(cfg.get("test_size", 0.2))
random_state = int(cfg.get("random_state", 42))
do_stratify = (pd.Series(y).nunique() > 1)
print(f"[INFO] Split params -> test_size={test_size} | random_state={random_state} | stratify={do_stratify}")

# Split com estratificação quando aplicável
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_size,
    random_state=random_state,
    stratify=y if do_stratify else None,
)
print(f"[INFO] X_train: {X_train.shape} | X_test: {X_test.shape}")

# Conferência da proporção da target no geral vs train/test
def _dist(s):
    vc = s.value_counts(dropna=False)
    pct = (vc / len(s) * 100).round(2)
    return pd.DataFrame({"count": vc, "pct": pct})

print("\n[CHECK] Distribuição da target — geral / train / test")
disp_overall = _dist(y)
disp_train   = _dist(y_train)
disp_test    = _dist(y_test)
print("• Geral:\n", disp_overall.to_string())
print("• Train:\n", disp_train.to_string())
print("• Test:\n", disp_test.to_string())

# Alerta simples de desbalanceamento (macro-level)
imbalance_threshold = 0.80  # ajuste se quiser
top_ratio = disp_overall["pct"].max() / 100.0
if top_ratio >= imbalance_threshold:
    print(f"[AVISO] Target potencialmente desbalanceada (classe majoritária ~{top_ratio:.1%}). "
          "Considere métricas robustas (F1, ROC-AUC), validação estratificada e/ou técnicas de balanceamento.")

# Sinalização de categorias raras (pode explodir OHE)
rare_threshold = 5  # frequência mínima
rare_report = {}
for c in cat_cols:
    cnt = X_train[c].value_counts(dropna=False)
    rare = cnt[cnt < rare_threshold]
    if len(rare) > 0:
        rare_report[c] = len(rare)
if rare_report:
    top_rare_cols = sorted(rare_report.items(), key=lambda kv: kv[1], reverse=True)[:10]
    print(f"[INFO] Colunas categóricas com categorias raras (<{rare_threshold} amostras) no train:")
    for col, n_rare in top_rare_cols:
        print(f" - {col}: {n_rare} categorias raras")
else:
    print("[INFO] Nenhuma categoria rara detectada no train com o limite atual.")


[INFO] Colunas numéricas: 62 | categóricas: 65 | ignoradas: 0
[INFO] Split params -> test_size=0.2 | random_state=42 | stratify=True
[INFO] X_train: (5634, 127) | X_test: (1409, 127)

[CHECK] Distribuição da target — geral / train / test
• Geral:
        count    pct
Churn              
no      5174  73.46
yes     1869  26.54
• Train:
        count    pct
Churn              
no      4139  73.46
yes     1495  26.54
• Test:
        count    pct
Churn              
no      1035  73.46
yes      374  26.54
[INFO] Colunas categóricas com categorias raras (<5 amostras) no train:
 - customerID: 5634 categorias raras


In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,PaymentMethod_len,PaymentMethod_word_count,PaymentMethod_has_error,PaymentMethod_has_cancel,PaymentMethod_has_premium,Churn_len,Churn_word_count,Churn_has_error,Churn_has_cancel,Churn_has_premium
0,7590-VHVEG,female,0,yes,no,1,no,no phone service,dsl,no,...,16,2,False,False,False,2,1,False,False,False
1,5575-GNVDE,male,0,no,no,34,yes,no,dsl,yes,...,12,2,False,False,False,2,1,False,False,False
2,3668-QPYBK,male,0,no,no,2,yes,no,dsl,yes,...,12,2,False,False,False,3,1,False,False,False
3,7795-CFOCW,male,0,no,no,45,no,no phone service,dsl,yes,...,25,3,False,False,False,2,1,False,False,False
4,9237-HQITU,female,0,no,no,2,yes,no,fiber optic,no,...,16,2,False,False,False,3,1,False,False,False


# ⚙️ Pré-processamento — One-Hot denso + escala opcional

Nesta etapa, preparamos `X` para treinamento aplicando **imputação**, **codificação categórica** e **padronização opcional**.
Mantemos o bloco **visível no N2** por transparência pedagógica: quem lê o notebook consegue enxergar claramente *como* os dados chegam ao modelo.

## O que acontece aqui

1. **Detecção dos conjuntos de colunas**
   - Usamos `num_cols` e `cat_cols` (derivados de `summarize_columns(X)`) para endereçar o tratamento adequado a cada tipo.

2. **Pipeline numérico**
   - `SimpleImputer(strategy="mean")` para preencher valores ausentes.
   - `StandardScaler()` **opcional** (controlado por `cfg["scale_numeric"]`), deixando as features numéricas com média 0 e desvio 1.
   - Este bloco é montado dinamicamente: se `scale_numeric=False`, o scaler é omitido.

3. **Pipeline categórico**
   - `SimpleImputer(strategy="most_frequent")` para preencher categorias ausentes.
   - `OneHotEncoder(handle_unknown="ignore", output denso)`. O código é **compatível** com versões antigas/novas do scikit-learn:
     - usa `sparse_output=False` quando disponível;
     - faz *fallback* para `sparse=False` em versões anteriores.

4. **ColumnTransformer**
   - Une os dois pipelines e aplica a transformação **apenas às colunas selecionadas**, descartando o restante (`remainder="drop"`).
   - Mantemos `verbose_feature_names_out=False` para preservar nomes legíveis nas colunas expandidas do One-Hot.

5. **Ajuste e transformação**
   - `fit` no `X_train` (usando `y_train` quando necessário) e `transform` no `X_train`/`X_test`.
   - Garantimos saída **densa** (arrays NumPy), apropriada para modelos que não aceitam matrizes esparsas.

6. **Diagnósticos rápidos**
   - Imprimimos:
     - `scale_numeric`, tamanho de `num_cols`/`cat_cols`,
     - quantidade total de **features transformadas**,
     - *preview* dos **primeiros nomes** de features,
     - `shapes` de `X_train_t`/`X_test_t`,
     - **memória estimada** (MB) dos arrays transformados.

## Observações importantes

- **Identificadores (IDs)** como `customerID` não devem compor `cat_cols` (explodem o One-Hot e não agregam sinal preditivo).  
  Remova-os de `X` *antes* do preprocess ou exclua-os da lista de categóricas.
- Para colunas categóricas com **alta cardinalidade**, considere alternativas como **hashing trick**, **target encoding** ou **CatBoost encoding** em versões futuras do template.
- Se a memória ficar alta, avalie:
  - reduzir cardinalidade (agrupar categorias raras),
  - trocar o encoder,
  - ou trabalhar com saída **esparsa** (e modelos que aceitem esparsidade).

## Próximo passo

- Encaixar o `preprocess` nos **Pipelines** dos modelos (Dummy, LogisticRegression, KNN, RandomForest, etc.) e seguir para o **seletor de modelos e hiperparâmetros (UI)**.


In [5]:
# === Pré-processamento (One-Hot denso + escala opcional) ===
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd

# Função compatível com versões antigas/novas do scikit-learn
def build_preprocess(numeric_cols, categorical_cols, scale_numeric=True):
    """
    Cria um ColumnTransformer com:
      - Numéricas: imputação média + (opcional) StandardScaler
      - Categóricas: imputação mais frequente + OneHotEncoder denso (compatível com versões)
    """
    # 1) OneHotEncoder compatível com sklearn >= 1.4 (sparse_output) e versões anteriores (sparse)
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

    # 2) Pipeline numérico
    num_steps = [("imputer", SimpleImputer(strategy="mean"))]
    if scale_numeric and len(numeric_cols) > 0:
        num_steps.append(("scaler", StandardScaler()))
    numeric_transformer = Pipeline(steps=num_steps)

    # 3) Pipeline categórico
    cat_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", ohe),
    ])

    # 4) ColumnTransformer
    ct = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_cols if len(numeric_cols) > 0 else []),
            ("cat", cat_transformer, categorical_cols if len(categorical_cols) > 0 else []),
        ],
        remainder="drop",
        verbose_feature_names_out=False,
    )
    return ct


# ---- Construção do pré-processador conforme config
scale_numeric = bool(cfg.get("scale_numeric", True))
preprocess = build_preprocess(num_cols, cat_cols, scale_numeric=scale_numeric)
print(f"[INFO] scale_numeric={scale_numeric} | num_cols={len(num_cols)} | cat_cols={len(cat_cols)}")

# ---- Ajuste no treino
preprocess.fit(X_train, y_train)

# ---- Tentativa de inspecionar nomes de features geradas
feat_names = None
try:
    feat_names = preprocess.get_feature_names_out()
    print(f"[INFO] Features transformadas: {len(feat_names)}")
    print("[INFO] Preview das primeiras 20 features:")
    for n in feat_names[:20]:
        print(" -", n)
except Exception as e:
    print(f"[WARN] Não foi possível obter nomes de features: {e}")

# ---- Transformação de treino e teste (densa)
X_train_t = preprocess.transform(X_train)
X_test_t  = preprocess.transform(X_test)

# Garante array denso (caso algum backend retorne esparso)
if hasattr(X_train_t, "toarray"):
    X_train_t = X_train_t.toarray()
if hasattr(X_test_t, "toarray"):
    X_test_t = X_test_t.toarray()

print(f"[INFO] Shapes transformados -> X_train_t: {X_train_t.shape} | X_test_t: {X_test_t.shape}")

# ---- (Opcional) Relato rápido de memória
def _mb(nbytes): 
    return f"{(nbytes or 0) / (1024**2):.2f} MB"

try:
    mem_train = X_train_t.nbytes if isinstance(X_train_t, np.ndarray) else None
    mem_test  = X_test_t.nbytes  if isinstance(X_test_t,  np.ndarray) else None
    print(f"[INFO] Memória estimada -> train={_mb(mem_train)} | test={_mb(mem_test)}")
except Exception:
    pass

# ---- (Opcional) DataFrame de features apenas para DEBUG/inspeção (cuidado com memória!)
# Ative só quando necessário; por padrão, mantemos como arrays numpy eficientes.
# if feat_names is not None:
#     X_train_df = pd.DataFrame(X_train_t, columns=feat_names, index=X_train.index)
#     X_test_df  = pd.DataFrame(X_test_t,  columns=feat_names, index=X_test.index)
#     display(X_train_df.head(3))


[INFO] scale_numeric=True | num_cols=62 | cat_cols=65
[INFO] Features transformadas: 5787
[INFO] Preview das primeiras 20 features:
 - SeniorCitizen
 - tenure
 - MonthlyCharges
 - TotalCharges
 - SeniorCitizen_was_missing
 - tenure_was_missing
 - MonthlyCharges_was_missing
 - TotalCharges_was_missing
 - customerID_was_missing
 - gender_was_missing
 - Partner_was_missing
 - Dependents_was_missing
 - PhoneService_was_missing
 - MultipleLines_was_missing
 - InternetService_was_missing
 - OnlineSecurity_was_missing
 - OnlineBackup_was_missing
 - DeviceProtection_was_missing
 - TechSupport_was_missing
 - StreamingTV_was_missing
[INFO] Shapes transformados -> X_train_t: (5634, 5787) | X_test_t: (1409, 5787)
[INFO] Memória estimada -> train=248.75 MB | test=62.21 MB


## 4) Seletor de modelos e hiperparâmetros (UI)

In [7]:
# === UI: Seletor de modelos + Hyperdrive (encapsulado no utils_data) ===
from utils.utils_data import n2_build_models_ui

n2_build_models_ui(
    preprocess=preprocess,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    models_dir=models_dir,
    reports_dir=reports_dir
)


Box(children=(VBox(children=(HBox(children=(HTML(value="<div class='lumen-title'>Seletor de modelos · Hyperdri…

## 5) Treino, avaliação e export de artefatos

In [ ]:

def collect_params_from_tab():
    return {name: {k: w.value for k, w in spec["params"].items()} for name, spec in MODEL_REGISTRY.items()}

def compute_and_plot(pipe, name, X_test, y_test):
    y_pred = pipe.predict(X_test)
    metrics = compute_metrics(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(5,4))
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred)).plot(ax=ax, colorbar=False)
    ax.set_title(f"Matriz de confusão — {name}")
    plt.show()
    try_plot_roc(pipe, X_test, y_test)
    print(f"[OK] {name}: accuracy={metrics['accuracy']:.4f} | f1={metrics['f1']:.4f}")
    return metrics

def train_and_eval(models_selected, params_by_model):
    results = {}
    for name, selected in models_selected.items():
        if not selected: 
            continue
        ModelClass = MODEL_REGISTRY[name]["class"]
        params = params_by_model.get(name, {})
        pipe = Pipeline(steps=[("preprocess", preprocess), ("clf", ModelClass(**params))])
        pipe.fit(X_train, y_train)
        metrics = compute_and_plot(pipe, name, X_test, y_test)
        results[name] = {"pipeline": pipe, "metrics": metrics, "params": params}
    return results

@out.capture()
def on_train_clicked(_):
    clear_output(wait=True)
    display(W.HTML("<h4>Treinando...</h4>"))
    selected = {name: chk.value for name, chk in model_checks.items()}
    params = collect_params_from_tab()
    results = train_and_eval(selected, params)

    if results:
        df_rank = pd.DataFrame([{"model": k, **v["metrics"], **{"params": v["params"]}} for k, v in results.items()])\
                    .sort_values(by=["f1", "accuracy"], ascending=False)
        display(W.HTML("<h4>Ranking (F1, depois Accuracy)</h4>")); display(df_rank)
        if cb_persist.value:
            for name, rec in results.items():
                persist_artifacts(name, rec["pipeline"], rec["metrics"], rec["params"], models_dir, reports_dir)
    else:
        print("[AVISO] Nenhum modelo selecionado.")

btn_train.on_click(on_train_clicked)
